# Tamil Audio → Tamil Text


In [10]:
!pip install gradio transformers soundfile

import gradio as gr
from transformers import pipeline

# Load the Whisper model
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3")

def transcribe(audio):
    if audio is None:
        return "No audio provided. Please record or upload audio."

    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(type="filepath"),
    outputs="text",
    title="Whisper Large v3 Demo",
    description="Transcribe audio using the OpenAI Whisper Large v3 model. Click the microphone button to record audio or upload a file."
)

iface.launch(debug=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Device set to use cuda:0


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://187b7d7801ec2ee964.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_featu

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://cbdf941f50a4d5dda5.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://3128b12eb54d8caf4c.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://67692acf1c9221ac06.gradio.live
Killing tunnel 127.0.0.1:7863 <> https://187b7d7801ec2ee964.gradio.live


# Tamil Text → English Translation

In [13]:
!pip install gradio transformers sentencepiece

import gradio as gr
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the NLLB model and tokenizer
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Limited set of sample languages
language_codes = {
    "English": "eng_Latn",
    "Tamil": "tam_Taml",
    "Hindi": "hin_Deva",
    "Spanish": "spa_Latn",
    "French": "fra_Latn",
    "German": "deu_Latn",
    "Chinese": "zho_Hans",
    "Japanese": "jpn_Jpan",
    "Arabic": "ara_Arab",
    "Russian": "rus_Cyrl"
}

def translate(text, source_lang, target_lang):
    if not text:
        return "No text provided. Please enter some text to translate."

    src_lang_code = language_codes[source_lang]
    tgt_lang_code = language_codes[target_lang]

    # Set source language by setting it as an attribute of the tokenizer
    tokenizer.src_lang = src_lang_code

    # Encode without the src_lang parameter
    encoded = tokenizer(text, return_tensors="pt")

    # Generate translation with the correct target language token
    translated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang_code),
        max_length=256
    )

    translation = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    return translation

iface = gr.Interface(
    fn=translate,
    inputs=[
        gr.Textbox(lines=5, placeholder="Enter text to translate here...", label="Input Text"),
        gr.Dropdown(list(language_codes.keys()), value="English", label="Source Language"),
        gr.Dropdown(list(language_codes.keys()), value="Tamil", label="Target Language")
    ],
    outputs=gr.Textbox(lines=5, label="Translated Text"),
    title="NLLB-200 Translation Demo",
    description="Translate text between selected languages using Facebook's NLLB-200-Distilled-600M model."
)

iface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8186e3011122fd7608.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://984454fcd2ea0d23aa.gradio.live
Killing tunnel 127.0.0.1:7864 <> https://8186e3011122fd7608.gradio.live


In [6]:
!pip install python-dotenv

# English Text → Image

In [16]:
import gradio as gr
import requests
import os
from dotenv import load_dotenv
from PIL import Image
import io

# Load environment variables
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")
if HF_TOKEN is None:
    raise ValueError("Hugging Face token not found. Please set HF_TOKEN in the .env file.")

# Hugging Face API Endpoint
IMAGE_API_URL = "https://api-inference.huggingface.co/models/black-forest-labs/FLUX.1-schnell"
headers = {"Authorization": f"Bearer {HF_TOKEN}"}

def generate_image(prompt, max_retries=5, retry_delay=5):
    """Query the FLUX.1-schnell model for text-to-image generation."""
    payload = {"inputs": prompt}
    for _ in range(max_retries):
        response = requests.post(IMAGE_API_URL, headers=headers, json=payload)
        if response.status_code == 200 and response.headers.get('Content-Type', '').startswith('image/'):
            return Image.open(io.BytesIO(response.content))
    return "Failed to generate image. Try again."

# Gradio Interface
iface = gr.Interface(
    fn=generate_image,
    inputs=gr.Textbox(label="Enter Image Description"),
    outputs=gr.Image(label="Generated Image"),
    title="FLUX.1-schnell - Text to Image Generator",
    description="Enter a text prompt to generate an AI image using FLUX.1-schnell model from Hugging Face.",
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e632846e9080214694.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# English Text → Further Text Generation

In [15]:
import gradio as gr
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")
if HF_TOKEN is None:
    raise ValueError("Hugging Face token not found. Please set HF_TOKEN in the .env file.")

# Hugging Face API Endpoint
TEXT_GEN_API_URL = "https://api-inference.huggingface.co/models/deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"
headers = {"Authorization": f"Bearer {HF_TOKEN}"}

def generate_text(prompt, max_retries=5, retry_delay=5):
    """Query the DeepSeek-R1-Distill-Qwen-32B model for text generation in English."""
    payload = {"inputs": prompt, "parameters": {"language": "en"}}
    for _ in range(max_retries):
        response = requests.post(TEXT_GEN_API_URL, headers=headers, json=payload)
        result = response.json()
        if isinstance(result, dict) and "generated_text" in result:
            return result["generated_text"]
        elif isinstance(result, list) and len(result) > 0 and "generated_text" in result[0]:
            return result[0]["generated_text"]
    return "Text generation failed. Try again."

# Gradio Interface
iface = gr.Interface(
    fn=generate_text,
    inputs=gr.Textbox(label="Enter Prompt"),
    outputs=gr.Textbox(label="Generated Text"),
    title="DeepSeek-R1-Distill-Qwen-32B - Text Generator",
    description="Enter a text prompt to generate AI-generated text in English using DeepSeek-R1-Distill-Qwen-32B model from Hugging Face.",
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://26ebaffa8902481b26.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
